In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/goodreadsbooks/books.csv


In [2]:
df = pd.read_csv("/kaggle/input/goodreadsbooks/books.csv", error_bad_lines=False)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 3350: expected 12 fields, saw 13\nSkipping line 4704: expected 12 fields, saw 13\nSkipping line 5879: expected 12 fields, saw 13\nSkipping line 8981: expected 12 fields, saw 13\n'


# Preprocessing

In [3]:
print("Count total NaN at each column in a DataFrame :", df.isnull().sum().sum())

df = df.drop(['isbn13', 'isbn', 'text_reviews_count', 'language_code'], axis = 1)
df.rename(columns={'  num_pages': 'pages'}, inplace=True)

df['full_book'] = df['title'] + ', ' + df['authors'] + ', ' + df['publisher'] + ', ' + df['average_rating'].astype(str) 

df = df.drop_duplicates(subset=['title'])
df = df.reset_index(drop=True)
df = df.drop(['bookID'], axis = 1)
df['bookID'] = df.index

df.head(5)

Count total NaN at each column in a DataFrame : 0


,title,authors,average_rating,pages,ratings_count,publication_date,publisher,full_book,bookID
0,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,652,2095690,9/16/2006,Scholastic Inc.,Harry Potter and the Half-Blood Prince (Harry ...,0
1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,870,2153167,9/1/2004,Scholastic Inc.,Harry Potter and the Order of the Phoenix (Har...,1
2,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,352,6333,11/1/2003,Scholastic,Harry Potter and the Chamber of Secrets (Harry...,2
3,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,435,2339585,5/1/2004,Scholastic Inc.,Harry Potter and the Prisoner of Azkaban (Harr...,3
4,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,2690,41428,9/13/2004,Scholastic,Harry Potter Boxed Set Books 1-5 (Harry Potte...,4


# Content-Based Recommendation System

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity 
tf = TfidfVectorizer()

def book_rec(Title):
    BookID = df[df.title == Title]['bookID'].values[0]
    vector_matrix = tf.fit_transform(df['full_book'])
    cs = cosine_similarity(vector_matrix)
    
    id_scores = list(enumerate(cs[BookID]))
    sort_scores = sorted(id_scores,key=lambda x:x[1],reverse=True)
    sort_scores = sort_scores[1:]
    display_scores = sort_scores[0:10]
    book_indices = [i[0] for i in display_scores]

    print('Similarity Check: ')
    res_list = [print(str(x[0])+':  '+str(x[1])+'%') for x in (display_scores)]
    print(res_list)
    
    return df['title'].iloc[book_indices] 

In [5]:
def book_rec1(Author):
    BookID = df[df.authors == Author]['bookID'].values[0]
    vector_matrix = tf.fit_transform(df['full_book'])
    cs = cosine_similarity(vector_matrix)

    id_scores = list(enumerate(cs[BookID]))
    sort_scores = sorted(id_scores,key=lambda x:x[1],reverse=True)
    sort_scores = sort_scores[1:]
    display_scores = sort_scores[0:10]
    book_indices = [i[0] for i in display_scores]
    
    print('Similarity Check: ')
    res_list = [print(str(x[0])+':  '+str(x[1])+'%') for x in (display_scores)]
    print(res_list) 
    
    return df['title'].iloc[book_indices]

In [6]:
df.tail(10)

,title,authors,average_rating,pages,ratings_count,publication_date,publisher,full_book,bookID
10338,O Cavalo e o Seu Rapaz (As Crónicas de Nárnia ...,C.S. Lewis/Pauline Baynes/Ana Falcão Bastos,3.92,160,207,8/15/2003,Editorial Presença,O Cavalo e o Seu Rapaz (As Crónicas de Nárnia ...,10338
10339,O Sobrinho do Mágico (As Crónicas de Nárnia #1),C.S. Lewis/Pauline Baynes/Ana Falcão Bastos,4.04,147,396,4/8/2003,Editorial Presença,O Sobrinho do Mágico (As Crónicas de Nárnia #...,10339
10340,A Viagem do Caminheiro da Alvorada (As Crónica...,C.S. Lewis/Pauline Baynes/Ana Falcão Bastos,4.09,176,161,9/1/2004,Editorial Presença,A Viagem do Caminheiro da Alvorada (As Crónica...,10340
10341,O Príncipe Caspian (As Crónicas de Nárnia #4),C.S. Lewis/Pauline Baynes/Ana Falcão Bastos,3.97,160,215,10/11/2003,Editorial Presença,O Príncipe Caspian (As Crónicas de Nárnia #4)...,10341
10342,Whores for Gloria,William T. Vollmann,3.69,160,932,2/1/1994,Penguin Books,"Whores for Gloria, William T. Vollmann, Pengui...",10342
10343,Expelled from Eden: A William T. Vollmann Reader,William T. Vollmann/Larry McCaffery/Michael He...,4.06,512,156,12/21/2004,Da Capo Press,Expelled from Eden: A William T. Vollmann Read...,10343
10344,You Bright and Risen Angels,William T. Vollmann,4.08,635,783,12/1/1988,Penguin Books,"You Bright and Risen Angels, William T. Vollma...",10344
10345,The Ice-Shirt (Seven Dreams #1),William T. Vollmann,3.96,415,820,8/1/1993,Penguin Books,"The Ice-Shirt (Seven Dreams #1), William T. Vo...",10345
10346,Poor People,William T. Vollmann,3.72,434,769,2/27/2007,Ecco,"Poor People, William T. Vollmann, Ecco, 3.72",10346
10347,Las aventuras de Tom Sawyer,Mark Twain,3.91,272,113,5/28/2006,Edimat Libros,"Las aventuras de Tom Sawyer, Mark Twain, Edima...",10347


In [7]:
# Test 1
book_rec('Molly Hatchet - 5 of the Best')

Similarity Check: 
27:  0.30936668522625443%
30:  0.23266645950935352%
28:  0.22891157953227054%
26:  0.22429123936124842%
3588:  0.186676158584865%
203:  0.17774408073239234%
6200:  0.1759254530526786%
2669:  0.14078941694784716%
4139:  0.13363231570811626%
4137:  0.13309248649220987%
[None, None, None, None, None, None, None, None, None, None]


27      Hatchet: A Guide for Using "Hatchet" in the Cl...
30         Hatchet Jobs: Writings on Contemporary Fiction
28      Guts: The True Stories behind Hatchet and the ...
26                             Hatchet (Brian's Saga  #1)
3588                               More Milly Molly Mandy
203     The Zen of CSS Design: Visual Enlightenment fo...
6200    Common Ground: The Water  Earth  and Air We Share
2669    The Secret of Fantasy Forest (Sweet Valley Kid...
4139    A Dance to the Music of Time: 2nd Movement (A ...
4137    A Dance to the Music of Time: 1st Movement (A ...
Name: title, dtype: object

In [8]:
# Test 2
book_rec1('Mark Twain')

Similarity Check: 
8832:  0.6081203647472484%
826:  0.5844012589121094%
825:  0.562455292031787%
9008:  0.544445532506407%
4937:  0.49012934208576403%
4939:  0.46616513332564413%
821:  0.4639120991798406%
824:  0.3736835755089112%
817:  0.37095029715905214%
9007:  0.3331008803032338%
[None, None, None, None, None, None, None, None, None, None]


8832                           Mark Twain: Selected Works
826                       The Autobiography of Mark Twain
825              The Complete Short Stories of Mark Twain
9008                    The Complete Essays of Mark Twain
4937          Mark Twain's Adventures of Huckleberry Finn
4939    Mark Twain's Own Autobiography: The Chapters f...
821                        Adventures of Huckleberry Finn
824     Mark Twain's Helpful Hints for Good Living: A ...
817     Huck Finn & Tom Sawyer among the Indians & Oth...
9007                                          Roughing It
Name: title, dtype: object